# Python Homework 2

A stochastic model for a car insurance company's total cost of damages from traffic accidents goes back to the work by Van der Lann and Louter, "A statistical model for the costs of passenger car traffic accidents", Journal of the Royal Statistical Society (1986).

For every $k=1,2,3\ldots$ we denote by the random variable $X_k$ the US dollar amount of a damage from a policy holder's traffic accident which will occur during the year 2019.

We assume that $X_1$, $X_2$,... is an i.i.d. (independent and identically distributed) sequence of exponentially distributed random variables with an average claim size of \$1,000 USD. The (random) total number of accidents $N$ in 2019 is expected to be Poisson distributed with 20 claims on average.

It is assumed that the number of accidents is independent of the US dollar amount of damages for each accident. That is, the random variable $N$ is independent of the random variables $X_1$, $X_2$,...

The total costs for the insurance company by the end of 2019 will thus be given by the <b>random sum</b> 

$$S_N := X_1 + X_2 + \dots + X_N = \sum_{k = 1}^{N} X_k.$$

Note that the total number $N$ of accidents is random!

The goal of the current exercise is to approximate 

* the expected total costs $$\mathbb{E}[S_N]$$ for the insurance company in 2019, and


* the probabilities that the total cost will not exceed $K$ USD, i.e., 

$$\mathbb{P}[S_N \leq K] \quad \text{for} \, K = \$20,000,\, \$40,000,\, \$60,000$$

via simulation.

As usual, we start with loading some packages:

In [16]:
import numpy as np
import math

<b>Step 1:</b><br>
First, write a function which simulates the random variable $S_N$. The output should just be a single scalar!

<i>Hint:</i> Use proper build-in functions from the <i>NumPy</i>-package in your code in order to sample from a Poisson distribution and from an Exponential distribution: 

In [19]:
def randomSum(averageClaimSize, averageNumberOfClaims):
    
    N = np.random.poisson(lam=averageNumberOfClaims,size=1)
    X = np.random.exponential(scale=averageClaimSize,size=N)
    sampleRandomSum=sum(X)
          
    return sampleRandomSum

In [20]:
## Test your function
randomSum(1000,20)

27087.147810952854

<b>Step 2:</b><br>Write a simulator function which uses the function <tt>randomSum()</tt> to simulate $M \in \mathbb{N}$ samples from the random variable $S_N$. The output should be an array of length $M$.

In [21]:
def simulator(averageClaimSize, averageNumberOfClaims, M):
    
    samples = []
    for i in range(M):
        samples.append(randomSum(averageClaimSize, averageNumberOfClaims))
        
    return samples

In [22]:
## Test your function
simulator(1000,20,10)

[19719.5245373161,
 34631.35178811719,
 19419.48983707379,
 14758.293510379803,
 20664.311721045142,
 10264.332325109424,
 30258.71789774433,
 30869.528330974652,
 18276.825091986728,
 24255.069075355917]

<b>Step 3:</b><br>As we will show in class, it holds via the so-called <b>Wald's Identity</b> that the expectation of the random sum $S_N$ is actually given by the formula

\begin{equation}
\mathbb{E}[S_N] = \mathbb{E}[N] \cdot \mathbb{E}[X_1] = 20 \cdot \$1,000 = \$20,000.
\end{equation}

Check via the empirical mean that

$$ \$20,000 = \mathbb{E}[S_N] \approx \frac{1}{M} \sum_{m=1}^M s^{(m)}_N$$

where $s^{(1)}_N, s^{(2)}_N, \ldots, s^{(M)}_N$ denote $M$ independent realizations (samples) from the random variable $S_N$. Use $M = 10, 100, 1000, 10000$ simulations.  

That is, write a function <tt>MCsimulation()</tt> which uses the function <tt>simulator()</tt> to compute the empirical mean. The output should be a real number in $\mathbb{R}_+$:

In [23]:
def MCsimulation(averageClaimSize, averageNumberOfClaims, M):
    
    SN = sum(simulator(averageClaimSize, averageNumberOfClaims, M))
    empricialMean = SN/M
    
    return empricialMean

In [24]:
## Test your function
MCsimulation(1000,20,10)

21969.37521971801

In [25]:
## Compute the absolute error
print(np.absolute(MCsimulation(1000, 20, 10)-20000))
print(np.absolute(MCsimulation(1000, 20, 100)-20000))
print(np.absolute(MCsimulation(1000, 20, 1000)-20000))
print(np.absolute(MCsimulation(1000, 20, 10000)-20000))
print(np.absolute(MCsimulation(1000, 20, 20000)-20000))
print(np.absolute(MCsimulation(1000, 20, 50000)-20000))

2828.2241224159334
1337.7050987785187
1.3781650175842515
122.63004561898197
16.660168592090486
17.729773250972357


<b>Step 4:</b><br>Recall from class that the desired probabilities $\mathbb{P}[S_N \leq K]$ for $K = \$20,000,\, \$40,000,\, \$60,000$ can be computed as expectations via an indicator function

$$ \mathbb{P}[S_N \leq K] = \mathbb{E}[1_{\{S_N \leq K\}}]$$

We use once more the empricial mean to approximate

$$ \mathbb{E}[1_{\{S_N \leq K\}}] \approx \frac{1}{M} \sum_{m=1}^M 1_{\{s^{(m)}_N \leq K \}}$$

with $M$ independent realizations (samples) from the random variable $S_N$ (again denoted by $s^{(1)}_N, s^{(2)}_N, \ldots, s^{(M)}_N$).

Write a function <tt>MCprobEstimation()</tt> which estimates the probabilities $\mathbb{P}[S_N \leq K]$ for $K = \$20,000,\, \$40,000,\, \$60,000$ as described based on $M$ simulations of $S_N$. The output should be a real number in [0,1]:

In [50]:
def MCprobEstimation(averageClaimSize, averageNumberOfClaims, K, M):
    
    rs = simulator(averageClaimSize, averageNumberOfClaims, M)
    
    empricialProb = 0
    for i in range(M):
        if( rs[i] <= K):
            empricialProb = empricialProb + 1/M
    
    
    return empricialProb

In [51]:
## Test your function
MCprobEstimation(1000, 20, 20000, 10)

0.7

Test your function for all $K = \$20,000,\, \$40,000,\, \$60,000$ with varying $M = 100, 1000, 10000$ simulations:

In [55]:
averageClaimSize = 1000
averageNumberOfClaims = 20
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 20000, 100))
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 20000, 1000))
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 20000, 10000))

0.46000000000000024
0.5460000000000004
0.5270999999999583


In [56]:
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 40000, 100))
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 40000, 1000))
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 40000, 10000))

1.0000000000000007
0.9980000000000008
0.9960999999999066


In [54]:
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 60000, 100))
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 60000, 1000))
print(MCprobEstimation(averageClaimSize, averageNumberOfClaims, 60000, 10000))

0
0.003
0.0032999999999999982
